## 调用chatgpt api方式启动

In [2]:
from openai import OpenAI

# Create a new OpenAI instance
llm = OpenAI(model_name="gpt-3.5-turbo") 
print(llm("introduce yourself"))

TypeError: OpenAI.__init__() got an unexpected keyword argument 'model_name'